In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import xgboost
from sklearn.model_selection import train_test_split

In [4]:
df = pd.read_excel('two_hundred_points_v0.3.xlsx', index_col=[0])

X_feature = df.iloc[:, 4:]
Y_height = pd.DataFrame(np.zeros((len(df), 2)), columns=['left', 'right'])
Y_label = df.loc[:, ['left', 'right']]

In [5]:
X_feature

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,y190,y191,y192,y193,y194,y195,y196,y197,y198,y199
0,0.689526,0.701995,0.713217,0.725686,0.738155,0.750623,0.761845,0.774314,0.786783,0.798005,...,0.004978,0.003734,0.002489,0.002489,0.002489,0.001245,0.001245,0.001245,0.001245,0.001245
1,0.220698,0.233167,0.244389,0.256858,0.268080,0.280549,0.291771,0.304239,0.315461,0.327930,...,0.027348,0.021133,0.016160,0.012431,0.008702,0.006216,0.003729,0.002486,0.001243,0.001243
2,0.301746,0.314214,0.326683,0.339152,0.352868,0.365337,0.377805,0.390274,0.402743,0.415212,...,0.022399,0.022399,0.022399,0.022399,0.019910,0.019910,0.019910,0.018666,0.014932,0.002489
3,0.451372,0.463840,0.476309,0.488778,0.500000,0.512469,0.524938,0.537406,0.549875,0.562344,...,0.039788,0.036058,0.033571,0.029841,0.026111,0.021137,0.018651,0.017407,0.014920,0.002487
4,0.371571,0.387781,0.403990,0.421446,0.437656,0.453865,0.470075,0.486284,0.503741,0.519950,...,0.024923,0.012461,0.007477,0.004985,0.004985,0.004985,0.004985,0.003738,0.002492,0.002492
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1243,0.610973,0.623441,0.623441,0.630923,0.643392,0.655860,0.668329,0.680798,0.693267,0.705736,...,0.002491,0.003737,0.004982,0.006228,0.007473,0.007473,0.009964,0.011210,0.008719,0.001246
1244,0.396509,0.408978,0.421446,0.435162,0.447631,0.460100,0.472569,0.485037,0.498753,0.511222,...,0.046026,0.042295,0.039807,0.036075,0.032343,0.028611,0.026123,0.022391,0.011196,0.001244
1245,0.139651,0.153367,0.168329,0.182045,0.197007,0.210723,0.225686,0.239401,0.254364,0.268080,...,0.098422,0.084718,0.069767,0.056063,0.042359,0.028654,0.018688,0.011213,0.006229,0.001246
1246,0.381546,0.395262,0.410224,0.423940,0.437656,0.451372,0.466334,0.480050,0.493766,0.508728,...,0.011210,0.008719,0.007474,0.006228,0.004982,0.004982,0.002491,0.002491,0.002491,0.001246


In [6]:
for idx, val in df.iterrows():
    y_left = val['y' + str(val['left'])]
    y_right = val['y' + str(val['right'])]
    Y_height.loc[idx] = {'left': y_left, 'right': y_right}
Y_height

,left,right
0,0.396999,0.273792
1,0.229976,0.384122
2,0.419354,0.333492
3,0.360577,0.272298
4,0.686621,0.687867
...,...,...
1243,0.308888,0.349990
1244,0.376920,0.306014
1245,0.313953,0.645348
1246,0.335069,0.524401


In [22]:
# Y_label.to_excel('test_true.xlsx')

In [7]:
test_idx = pd.read_csv('testset_index.csv', index_col=0)["test index"].values.tolist()

x_train = X_feature[~X_feature.index.isin(test_idx)]
x_test  = X_feature[X_feature.index.isin(test_idx)]

y_train = Y_label[~Y_label.index.isin(test_idx)]
y_test  = Y_label[Y_label.index.isin(test_idx)]

height_train = Y_height[~Y_label.index.isin(test_idx)]
height_test  = Y_height[Y_label.index.isin(test_idx)]

In [25]:
height_test

,left,right
28,0.255674,0.299114
30,0.464280,0.331095
42,0.288551,0.383076
53,0.278049,0.331425
99,0.304371,0.334187
...,...,...
975,0.672062,0.677049
982,0.379504,0.286183
988,0.642958,0.665387
1031,0.373335,0.261334


In [8]:
model_L = xgboost.XGBRegressor()
model_L.fit(x_train, y_train['left'])

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [32]:
left_pred = model_L.predict(x_test)
height_left_pred = height_test['left'].copy()
for idx, val in enumerate(left_pred):
    height_left_pred.iloc[idx] = x_test.iloc[idx]['y' + str(int(round(val)))]

In [33]:
height_left_pred

28      0.239539
30      0.453078
42      0.337057
53      0.290462
99      0.295675
          ...   
975     0.657099
982     0.373282
988     0.687816
1031    0.373335
1034    0.656116
Name: left, Length: 100, dtype: float64

In [46]:
left_error = np.abs(height_test['left'] - height_left_pred) / height_test['left'] * 100
print(np.quantile(left_error, [0, 0.95, 0.96, 0.97, 0.98, 0.99, 1]))
left_error.describe()

[ 0.         12.23687424 14.0819376  16.82526525 17.36710506 20.28633103
 21.07843137]


count    100.000000
mean       3.488148
std        4.342445
min        0.000000
25%        0.000000
50%        2.396911
75%        4.480461
max       21.078431
Name: left, dtype: float64

In [36]:
model_R = xgboost.XGBRegressor()
model_R.fit(x_train, y_train['right'])

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [37]:
right_pred = model_R.predict(x_test)

In [41]:
right_pred = model_R.predict(x_test)
height_right_pred = height_test['right'].copy()
for idx, val in enumerate(right_pred):
    height_right_pred.iloc[idx] = x_test.iloc[idx]['y' + str(int(round(val)))]

In [47]:
right_error = np.abs(height_test['right'] - height_right_pred) / height_test['right'] * 100
print(np.quantile(right_error, [0, 0.95, 0.96, 0.97, 0.98, 0.99, 1]))
right_error.describe()

[ 0.         15.78149746 16.00747175 16.47651718 16.91632969 21.71281979
 34.06113537]


count    100.000000
mean       4.454854
std        5.237180
min        0.000000
25%        0.874398
50%        3.386288
75%        5.490612
max       34.061135
Name: right, dtype: float64

In [18]:
model_L = xgboost.XGBRegressor()
model_L.fit(X_feature, Y_label['left'])

model_R = xgboost.XGBRegressor()
model_R.fit(X_feature, Y_label['right'])

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [19]:
model_L.save_model('./xgb_model/outline_left_v0.4.model')
model_R.save_model('./xgb_model/outline_right_v0.4.model')